**Create pseudo labels during the semi-supervised phase.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

sys.path.append('/content/drive/MyDrive/Wound_tissue_segmentation')
sys.path.append('/content/drive/MyDrive/Wound_tissue_segmentation/utils')
sys.path.append('/content/drive/MyDrive/Wound_tissue_segmentation/wound_lib')

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu
import cv2
import numpy as np
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils import metrics, losses, base
import random
import matplotlib.pyplot as plt
import os
from copy import deepcopy
from datetime import datetime
import torch.nn.functional as F

%matplotlib inline

## Dataloader

In [ ]:
class Dataset(BaseDataset):
    """CamVid Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """

    def __init__(
            self,
            list_IDs,
            images_dir,
            preprocessing=None,
            resize=(False, (256, 256)), # To resize, the first value has to be True
            n_classes:int=4,
    ):
        self.ids = list_IDs
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]

        self.preprocessing = preprocessing
        self.resize = resize
        self.n_classes = n_classes

    def __getitem__(self, i):

        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.resize[0]:
            image = cv2.resize(image, self.resize[1], interpolation=cv2.INTER_NEAREST)

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']

        return image

    def __len__(self):
        return len(self.ids)

## Augmentation

In [ ]:
def get_training_augmentation():
    train_transform = [

        albu.OneOf(
            [
                albu.HorizontalFlip(p=0.5),
                albu.VerticalFlip(p=0.5),
            ],
            p=0.8,
        ),

        albu.OneOf(
            [
                albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0, p=0.1, border_mode=0), # scale only
                albu.ShiftScaleRotate(scale_limit=0, rotate_limit=30, shift_limit=0, p=0.1, border_mode=0), # rotate only
                albu.ShiftScaleRotate(scale_limit=0, rotate_limit=0, shift_limit=0.1, p=0.6, border_mode=0), # shift only
                albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=30, shift_limit=0.1, p=0.2, border_mode=0), # affine transform
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.Perspective(p=0.2),
                albu.GaussNoise(p=0.2),
                albu.Sharpen(p=0.2),
                albu.Blur(blur_limit=3, p=0.2),
                albu.MotionBlur(blur_limit=3, p=0.2),
            ],
            p=0.5,
        ),

        albu.OneOf(
            [
                albu.CLAHE(p=0.25),
                albu.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.25),
                albu.RandomGamma(p=0.25),
                albu.HueSaturationValue(p=0.25),
            ],
            p=0.3,
        ),

    ]

    return albu.Compose(train_transform, p=0.9) # 90% augmentation probability


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        # albu.PadIfNeeded(512, 512)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

## Parameters

In [ ]:
# Parameters
BASE_MODEL = 'MiT+pscse'
ENCODER = 'mit_b3'
ENCODER_WEIGHTS = 'imagenet'
BATCH_SIZE = 16
n_classes = 4
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LR = 0.0001 # learning rate
EPOCHS = 500
WEIGHT_DECAY = 1e-5
SAVE_WEIGHTS_ONLY = True
RESIZE = (False, (256,256)) # if resize needed
TO_CATEGORICAL = True
SAVE_BEST_MODEL = True
SAVE_LAST_MODEL = False

PERIOD = 10 # periodically save checkpoints
RAW_PREDICTION = False # if true, then stores raw predictions (i.e. before applying threshold)
RETRAIN = False

# For early stopping
EARLY_STOP = True # True to activate early stopping
PATIENCE = 50 # for early stopping

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


## Load model

In [ ]:
save_dir_pred_root = '/content/drive/MyDrive/Wound_tissue_segmentation/predictions'
os.makedirs(save_dir_pred_root, exist_ok = True)

# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    # aux_params=aux_params,
    classes=n_classes,
    activation=ACTIVATION,
    decoder_attention_type='pscse',
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

model.to(DEVICE)

# Optimizer
optimizer = torch.optim.Adam([
    dict(params=model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY),
])

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                              factor=0.1,
                              mode='min',
                              patience=10,
                              min_lr=0.00001,
                              verbose=True,
                              )

# Model
model_name = 'MiT+pscse_padded_mit_b3_2023-11-24_11-58-23_selfSupervised'       # <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< pay attention
print(model_name)

# Checkpoint directory
checkpoint_loc = '/content/drive/MyDrive/Wound_tissue_segmentation/checkpoints/' + model_name

# =================================== Inference ================================
# Load model====================================================================
checkpoint = torch.load(os.path.join(checkpoint_loc, 'best_model.pth'))
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
"""
IMPORTANT:
Uncomment only for this run. Here, creating duplicates of unsupervised images. Images will be
deleted after each run.

In next runs, DO NOT run this cell.
"""

import shutil

copy_src = '/content/drive/MyDrive/Wound_tissue_segmentation/Dataset/FUWound_mmseg_all_in_one_cropped_padded/unsupervised/images'

copy_dst = '/content/drive/MyDrive/Wound_tissue_segmentation/Dataset/FUWound_mmseg_all_in_one_cropped_padded/unsupervised/images_del_'

os.makedirs(copy_dst, exist_ok=True)

names = os.listdir(copy_src)

for name in names:
  shutil.copy(os.path.join(copy_src, name), os.path.join(copy_dst, name))

In [ ]:
x_test_dir = '/content/drive/MyDrive/Wound_tissue_segmentation/Dataset/FUWound_mmseg_all_in_one_cropped_padded/unsupervised/images_del_'

print('No. of test images: ', len(os.listdir(x_test_dir)))

In [ ]:
# Delete images from previous phase
# Replace existing images using phase-1 images
txt_phase1 = 'MiT+pscse_padded_mit_b3_2023-11-24_11-58-23_selfSupervised_unsup_train.txt'
dir_txt_phase1 = '/content/drive/MyDrive/Wound_tissue_segmentation/texts/' + txt_phase1 # full directory
dir_annotations_phase1 = '/content/drive/MyDrive/Wound_tissue_segmentation/predictions/' + 'MiT+pscse_padded_mit_b3_2023-11-24_11-58-23_selfSupervised'

# First, read the text file
with open(dir_txt_phase1, "r") as f:
  phase_names = f.readlines()

phase_names = [name.split('\n')[0] for name in phase_names] # remove \n (newline)

# First, delete existing annotations
for name in phase_names:
  if os.path.exists(os.path.join(x_test_dir, name)): os.remove(os.path.join(x_test_dir, name))

list_IDs_test = os.listdir(x_test_dir)

print('No. of test images: ', len(list_IDs_test))

In [ ]:
# Test dataloader ==============================================================
test_dataset = Dataset(
    list_IDs_test,
    x_test_dir,
    preprocessing=get_preprocessing(preprocessing_fn),
    resize=(RESIZE),
    n_classes=n_classes,
)

test_dataloader = DataLoader(test_dataset,
                            batch_size=1,
                            shuffle=False,
                            num_workers=6)

# Prediction ===================================================================
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import confusion_matrix
import scipy.io as sio

import warnings
warnings.filterwarnings("ignore")

save_pred = True
threshold = 0.5
ep = 1e-6
raw_pred = []

HARD_LINE = True

# Save directory
save_dir_pred = '/content/drive/MyDrive/Wound_tissue_segmentation/predictions/' + model_name + '_phase2'
if not os.path.exists(save_dir_pred): os.makedirs(save_dir_pred)

save_dir_pred_palette = '/content/drive/MyDrive/Wound_tissue_segmentation/predictions_palettte/' + model_name + '_phase2'
if not os.path.exists(save_dir_pred_palette): os.makedirs(save_dir_pred_palette)

iter_test_dataloader = iter(test_dataloader)

palette = [[128, 128, 128], [255, 0, 0], [0, 255, 0], [0, 0, 255]]

for enu, i in enumerate(range(len(list_IDs_test))):

    name = os.path.splitext(list_IDs_test[i])[0] # remove extension

    print('Processing:', enu, name)

    # Image-wise mean of metrics
    i_mp, i_mr, i_mdice, i_miou = [], [], [], []

    image = next(iter_test_dataloader) # get image and mask as Tensors

    # Note: Image shape: torch.Size([1, 3, 512, 512]) and mask shape: torch.Size([1, 1, 512, 512])

    pr_mask = model.predict(image.to(DEVICE)) # Move image tensor to gpu

    # Convert from onehot
    # gt_mask = torch.argmax(gt_mask_, dim=1)
    if TO_CATEGORICAL:
        pr_mask = torch.argmax(pr_mask, dim=1)

    # pr_mask = torch.argmax(pr_mask, dim=1)

    # Move to CPU and convert to numpy
    pred = pr_mask.squeeze().cpu().numpy()

    # Save raw prediction
    if RAW_PREDICTION: raw_pred.append(pred)

    # Modify prediction based on threshold
    # pred = (pred >= threshold) * 1

    # Save prediction as png
    if save_pred:
        "Uncomment for non-palette"
        cv2.imwrite(os.path.join(save_dir_pred, list_IDs_test[i]), np.squeeze(pred).astype(np.uint8))

        "Uncomment for palette"
        # Palette prediction
        pal_pred = np.squeeze(pred).astype(np.uint8)
        pal_pred = Image.fromarray(pal_pred)
        pal_pred = pal_pred.convert("P")
        pal_pred.putpalette(np.array(palette, dtype=np.uint8))

        # Store
        pal_pred.save(os.path.join(save_dir_pred_palette, list_IDs_test[i]))